#  Train GPT-2 Text-Generating Model for Question Generation

Adapted from [Max Woolf](http://minimaxir.com) ([this GitHub repository](https://github.com/minimaxir/gpt-2-simple), [blog post](https://minimaxir.com/2019/09/howto-gpt2/))

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
import shutil
import json
import tensorflow as tf
from gpt_2_simple.src import model
from tqdm import tqdm

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
#gpt2.download_gpt2(model_name="355M")
#!nvidia-smi

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [2]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')   #force_remount=True

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def is_mounted():
    """Checks if the Google Drive is mounted."""
    assert os.path.isdir('/content/drive'), "You must mount first using mount_gdrive()"

def get_file(file_name):
  is_mounted()
  file_path = "NLU_Project/SummQG/data/squad/contexts_sentences_questions_answers/"+file_name 
  print("File path:", file_path)
  shutil.copyfile("/content/drive/My Drive/" + file_path, file_name)

In [0]:
file_name = "squad_csqa_train.txt"

In [6]:
get_file(file_name)

File path: NLU_Project/SummQG/data/squad/contexts_sentences_questions_answers/squad_csqa_train.txt


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
# Previous Models Trained
# squad_csqa-15000-0.0001
# squad_csqa-20000-0.0001
# squad_csqa-25000-0.00005
# squad_csqa-30000-0.00001 #plateau 0.48
# "squad_csqa-35000-0.00001" #plateau 0.49 NO beeter than 30000-0.00001

run_name = "squad_csqa-35000-0.00001"
steps = 10000
lr = 0.00001

In [0]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name="squad_csqa-25000-0.00005",
              restore_from = "latest",
              steps=steps,
              learning_rate = lr,
              overwrite = False,
              run_name=run_name,
              print_every=100,
              sample_every=steps,
              save_every=steps
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/squad_csqa-25000-0.00005/model-5000
INFO:tensorflow:Restoring parameters from models/squad_csqa-25000-0.00005/model-5000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:24<00:00, 84.59s/it]


dataset has 18580236 tokens
Training...
[100 | 96.96] loss=0.70 avg=0.70
[200 | 185.10] loss=0.75 avg=0.73
[300 | 273.23] loss=0.42 avg=0.62
[400 | 361.36] loss=0.26 avg=0.53
[500 | 449.51] loss=0.40 avg=0.51
[600 | 537.61] loss=0.53 avg=0.51
[700 | 625.67] loss=0.49 avg=0.51
[800 | 713.79] loss=0.11 avg=0.46
[900 | 801.84] loss=0.68 avg=0.48
[1000 | 889.88] loss=0.31 avg=0.46
[1100 | 977.91] loss=0.72 avg=0.49
[1200 | 1065.98] loss=0.52 avg=0.49


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name=run_name)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
run_name = "squad_csqa-15000-0.0001"

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
#checkpoint_dir = "checkpoint"
#checkpoint_path = os.path.join(checkpoint_dir, run_name)
#print(checkpoint_path)
#hparams = model.default_hparams()
#with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
#      hparams.override_from_dict(json.load(f))
#ckpt = tf.train.latest_checkpoint(checkpoint_path)

'checkpoint/nqa-20000-0.00002/model-5000'

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, checkpoint_dir="/content/checkpoint", run_name=run_name)

Loading checkpoint /content/checkpoint/squad_sq-30000-0.000001/model-5000
INFO:tensorflow:Restoring parameters from /content/checkpoint/squad_sq-30000-0.000001/model-5000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, run_name=run_name)

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
text_file = "squad_test_set.txt"

In [0]:
def squad_couplet_data_processing(data_file):
  contexts = []
  gold_questions = []
  with open(data_file, "r") as infile:
    text = infile.read()

  couplet = text.split("<|endoftext|>")
  for i in couplet:
    if i == "" or i == "\n":
      pass
    else:
      c_q = i.split("[question]")
      contexts.append(c_q[0]+"[question] ")
      gold_questions.append(c_q[1][:-1])
  return contexts, gold_questions

In [0]:
contexts, gold_questions = squad_couplet_data_processing("squad_test_set.txt")

In [0]:
len(gold_questions)

NameError: ignored

In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
import shutil
import json
import tensorflow as tf
from gpt_2_simple.src import model
from tqdm import tqdm

run_name = "squad_sq-30000-0.000001"
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, checkpoint_dir="/content/checkpoint", run_name=run_name)

text_file = "squad_test_set.txt"

def squad_couplet_data_processing(data_file):
  contexts = []
  gold_questions = []
  with open(data_file, "r") as infile:
    text = infile.read()

  couplet = text.split("<|endoftext|>")
  for i in couplet:
    if i == "" or i == "\n":
      pass
    else:
      c_q = i.split("[question]")
      contexts.append(c_q[0]+"[question] ")
      gold_questions.append(c_q[1][:-1])
  return contexts, gold_questions

contexts, gold_questions = squad_couplet_data_processing("squad_test_set.txt")

############################
generated_questions = {}
i = 400

for context in tqdm(contexts[i:i+50]):
  q = gpt2.generate(sess,
              run_name = run_name,
              length=50,
              temperature=0.5,
              top_k = 50, 
              top_p = 0.9,
              prefix= context,
              nsamples=10,
              batch_size=10,
              include_prefix = False,
              truncate = "<|endoftext|>",
              return_as_list=True
              )
  generated_questions[i] = q
  i += 1

output_file = "squad_sq_"+str(i)+".json"
with open(output_file, "w") as json_file:
  json.dump(generated_questions, json_file)
shutil.copyfile(output_file, "/content/drive/My Drive/NLU_Project/SummQG/results/squad_sq/" + output_file)

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loading checkpoint /content/checkpoint/squad_sq-30000-0.000001/model-5000
INFO:tensorflow:Restoring parameters from /content/checkpoint/squad_sq-30000-0.000001/model-5000


  0%|          | 0/50 [00:00<?, ?it/s]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


 88%|████████▊ | 44/50 [13:52<02:39, 26.55s/it]

######################################################

In [0]:
!ps -aux|grep python

root          24  0.1  0.3 484696 101572 ?       Sl   20:20   0:11 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root        1564 89.8 76.1 40488844 20382048 ?   Ssl  21:26  52:35 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-da3e1bb6-f0a2-45c3-a1ac-d20ae5151906.json
root        2815  0.0  0.0  39196  6496 ?        S    22:24   0:00 /bin/bash -c ps -aux|grep python
root        2817  0.0  0.0   5708   464 ?        D    22:24   0:00 grep python


In [0]:
!kill -9 1564

In [0]:
gpt2.generate(sess,
              run_name = run_name,
              length=50,
              temperature=0.4,
              top_k = 40, 
              top_p = 0.9,
              prefix= pre,
              nsamples=10,
              batch_size=10,
              include_prefix = False,
              truncate = "<|endoftext|>"
              )

 WHO IS ANTONY?

 WHO IS ANTONY?

 WHO IS ANTONY?

 WHO IS ANTONY?

 WHO IS ANTONY?

 WHO IS ANTONY?

 WHO IS ANTONY?

 WHO IS ANTONY?

 WHO IS ANTONY?

 WHO IS ANTONY?



In [0]:
ctx = input("Summary: ")

Summary:  Mark Hunter (Slater), a high school student in a sleepy suburb of Phoenix, Arizona, starts an FM pirate radio station that broadcasts from the basement of his parents' house. Mark is a loner, an outsider, whose only outlet for his teenage angst and aggression is his unauthorized radio station. His pirate station's theme song is "Everybody Knows" by Leonard Cohen and there are glimpses of cassettes by such alternative musicians as The Jesus and Mary Chain, Camper Van Beethoven, Primal Scream, Soundgarden, Ice-T, Bad Brains, Concrete Blonde, Henry Rollins, and The Pixies. By day, Mark is seen as a loner, hardly talking to anyone around him; by night, he expresses his outsider views about what is wrong with American society. When he speaks his mind about what is going on at his school and in the community, more and more of his fellow students tune in to hear his show. Nobody knows the true identity of "Hard Harry" or "Happy Harry Hard-on," as Mark refers to himself, until Nora D

In [0]:
ans = input("Answer: ")
qst = "[question]"
pre = '<|startoftext|>\n[summary] ' + ctx + "\n[answer]" + ans + "\n" + qst

Answer: Mark is a high school student in Phoenix.


In [0]:
gpt2.generate(sess,
              run_name = run_name,
              length=50,
              temperature=0.1,
              top_k = 40, 
              top_p = 0.9,
              prefix= pre,
              nsamples=10,
              batch_size=1,
              include_prefix = False,
              truncate = "<|endoftext|>"
              )

 What is Mark Hunter's hometown?

 What is Mark Hunter's hometown?

 What is Mark Hunter's hometown?

 What is Mark's hometown?

 What is Mark's hometown?

 What is Mark Hunter's hometown?

 What is Mark Hunter's hometown?

 What is Mark Hunter's hometown?

 What is Mark Hunter's hometown?

 What is Mark Hunter's hometown?



For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [0]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.